### This implementation of transform data taken from this link:

####  https://github.com/FrankBGao/read_xes

## Importantly, this implementation is very slow for large files (more than 200 MB), because it contains python-dictionaries and many loops

### Import library and activate moduls

In [ ]:
import xmltodict
import pandas as pd
from json import dumps, loads

In [ ]:
class From_XES_to_CSV():
    
    def __init__(self, path_to_log):
        self.path = path_to_log
        self.xml_string = open(self.path, mode='r').read()
        
    @staticmethod
    def get_one_event_dict(one_event, case_name,data_types):

        one_event_attri = list(one_event.keys())

        one_event_dict = {}
        for i in data_types:
            if i in one_event_attri:
                if type(one_event[i]) == list:
                    for j in one_event[i]:
                        one_event_dict[j['@key']] = j['@value']
                else:
                    one_event_dict[one_event[i]['@key']] = one_event[i]['@value']
        one_event_dict['case_name'] = case_name
        return one_event_dict


    def gain_one_trace_info(cls, one_trace, data_types):
        # for the attributer
        one_trace_attri = list(one_trace.keys())
        one_trace_attri_dict = {}

        for i in data_types:
            if i in one_trace_attri:
                if type(one_trace[i]) == list:
                    for j in one_trace[i]:
                        one_trace_attri_dict[j['@key']] = j['@value']
                else:
                    one_trace_attri_dict[one_trace[i]['@key']] = one_trace[i]['@value']

        # for event seq
        one_trace_events = []
        if type(one_trace['event']) == dict:
            one_trace['event'] = [one_trace['event']]

        for i in one_trace['event']:
            inter_event = cls.get_one_event_dict(i, one_trace_attri_dict['concept:name'],data_types)
            one_trace_events.append(inter_event)

        return one_trace_attri_dict,one_trace_events


    def gain_log_info_table(cls):
        data_types = ['string', 'int', 'date', 'float', 'boolean', 'id']

        log_is = xmltodict.parse(cls.xml_string)
        log_is = loads(dumps(log_is))

        traces = log_is['log']['trace']

        trace_attri = []
        trace_event = []
        j = 0
        for i in traces:
            inter = cls.gain_one_trace_info(i,data_types)
            trace_attri.append(inter[0])
            trace_event = trace_event + inter[1]
            j = j +1

        return pd.DataFrame(trace_attri), pd.DataFrame(trace_event)

## Set path to log-file

In [ ]:
path = 'C:/Users/username/Documents/ProcessLog.xes'

## Get data as DataFrames

In [ ]:
%%time
trace_attri,trace_event = From_XES_to_CSV(path).gain_log_info_table()

## Save to csv-file

In [ ]:
trace_attri.to_csv("Cases_from_log.csv", index=False, sep=";")

In [ ]:
trace_event.to_csv("Events_from_log.csv", index=False, sep=";")